In [4]:
import os
import json
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm 
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [5]:
def process(df):
    ranked_sentences = []
    df['sentences'] = df['sentences'].apply(eval)
    df['judge-opinion'] = df['judge-opinion'].apply(eval)
    for i, row in tqdm(df.iterrows(),total=len(df)):
        s1 = df.loc[i]['sentences']
        judge_opinion=''
        for j in df.loc[i]['judge-opinion']:
            judge_opinion=judge_opinion+j
        s2 = judge_opinion
        e1 = model.encode(s1)
        e2 = model.encode(s2)
        cos_sim = util.cos_sim(e2, e1)
        s1 = np.asarray(s1)
        ranks = s1[np.argsort(-1*cos_sim[0].numpy())]
        ranked_sentences.append(ranks.tolist())
    
    df['ranked-sentences'] = ranked_sentences
    return df


In [6]:
base_path = "./data/summary/"
write_path = "./data/summary/"
files = os.listdir(base_path)

df=pd.read_csv(f"{base_path}/data.csv")

final_df=process(df)

final_df.to_csv(f"{write_path}/data_ranked.csv",index=False)

  0%|          | 0/1000 [00:00<?, ?it/s]